In [1]:
import sys; sys.path.append('../../wwa'); from wwa import *

## Extract gridded data for model fitting

In [3]:
fl = glob.glob("../../00_WWA_project_folder/ephemeral/Asia_Heatwave/obs/obs_fields/*.nc")

domain = {"EA" : [118,130,4,20], "WA" : [30,50,25,40]}

In [49]:
# ERA5
for fnm in glob.glob("../../00_WWA_project_folder/ephemeral/Asia_Heatwave/obs/obs_fields/era5*.nc"):
    
    xn,xx,yn,yx = domain[fnm[-5:-3]]
    
    da = xr.open_dataset(fnm).sel(lon = slice(xn,xx), lat = slice(yn,yx)).tmax
    da = da.assign_coords(time = da.time.dt.year).rename(time = "year")
    
    # mask off the sea
    rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
    da = da.where(rm)
    
    # flatten & save as .csv for transfer to R
    da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas().to_csv("data/"+fnm.split("/")[-1].split("_")[0]+"_"+fnm[-5:-3]+"_flattened.csv")
    
    # save map for easier reconstruction
    da.mean("year").to_netcdf("data/"+fnm.split("/")[-1].split("_")[0]+"_tmplt.nc")

In [50]:
# CPC
for fnm in glob.glob("../../00_WWA_project_folder/ephemeral/Asia_Heatwave/obs/obs_fields/cpc*.nc"):
    
    xn,xx,yn,yx = domain[fnm[-5:-3]]
    
    da = xr.open_dataset(fnm).sel(lon = slice(xn,xx), lat = slice(yn,yx)).tmax
    da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

    # mask off the sea
    rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
    da = da.where(rm)
    
    # flatten & save as .csv for transfer to R
    da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas().to_csv("data/"+fnm.split("/")[-1].split("_")[0]+"_"+fnm[-5:-3]+"_flattened.csv")
    
    # save map for easier reconstruction
    da.mean("year").to_netcdf("data/"+fnm.split("/")[-1].split("_")[0]+"_tmplt.nc")

In [51]:
# MSWX
for fnm in glob.glob("../../00_WWA_project_folder/ephemeral/Asia_Heatwave/obs/obs_fields/msw*.nc"):
    
    xn,xx,yn,yx = domain[fnm[-5:-3]]
    
    da = xr.open_dataset(fnm).sel(lon = slice(xn,xx), lat = slice(yn,yx)).air_temperature
    da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

    # mask off the sea
    rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
    da = da.where(rm)
    
    # flatten & save as .csv for transfer to R
    da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas().to_csv("data/"+fnm.split("/")[-1].split("_")[0]+"_"+fnm[-5:-3]+"_flattened.csv")
    
    # save map for easier reconstruction
    da.mean("year").to_netcdf("data/"+fnm.split("/")[-1].split("_")[0]+"_tmplt.nc")